In [1]:
from ConectionSQL import SQL
import pandas as pd

sql=SQL('Netfeira','sqlserver','MOINHO','192.168.0.252')

conectando=sql.ConexaoSQL()

pd.set_option('display.max_columns',None)

In [2]:
querys={
    
    'Carteira':
    
    """
    
    WITH TabEvento (cd_emp,cd_clien,nu_ped,dt_emissao,dt_fatu) AS (
    SELECT cd_emp,cd_clien,nu_ped,CAPV,
    CASE WHEN FATU IS NULL THEN CAPV ELSE FATU END AS FATU
    FROM (
    SELECT cd_fila,cd_emp,cd_clien,nu_ped,
    CONVERT(DATETIME,CAST(dt_criacao AS DATE),101) AS dt_criacao
    FROM evento
    WHERE cd_fila IN('FATU','CAPV'))evento
    PIVOT(MAX(dt_criacao) FOR cd_fila IN([CAPV],[FATU]))C),

    TabPedido (nu_ped,cd_clien,cd_vend,dt_fatu,vl_total) AS (

    SELECT ped_vda.nu_ped,ped_vda.cd_clien,cd_vend,TabEvento.dt_fatu,
    SUM(it_pedv.qtde_unid_vda*it_pedv.vl_unit_vda) AS vl_total
    FROM ped_vda
    INNER JOIN it_pedv ON ped_vda.nu_ped=it_pedv.nu_ped AND it_pedv.situacao IN ('FA','AB')
    INNER JOIN tp_ped ON ped_vda.tp_ped=tp_ped.tp_ped AND tp_ped.estat_com=1
    INNER JOIN TabEvento ON ped_vda.nu_ped=TabEvento.nu_ped
    GROUP BY ped_vda.nu_ped,ped_vda.cd_clien,cd_vend,TabEvento.dt_fatu),

    TabCarteira (cd_clien,cd_vend,prioritario,dt_min,dt_max,contagem,dt_atual) AS (

    SELECT vend_cli.cd_clien,vend_cli.cd_vend,prioritario,
    CASE WHEN dt_min IS NULL THEN CONVERT(DATETIME,CAST(cliente.dt_cad AS DATE),101)
    ELSE dt_min END AS dt_min,
    CASE WHEN dt_max IS NULL THEN CONVERT(DATETIME,CAST(cliente.dt_cad AS DATE),101)
    ELSE dt_max END dt_max,
    COUNT(vend_cli.cd_clien)OVER(PARTITION BY vend_cli.cd_clien ORDER BY vend_cli.cd_clien) AS contagem,
    CONVERT(DATETIME,CAST(GETDATE() AS DATE),101) AS dt_atual
    FROM (
    SELECT cd_clien,cd_vend,prioritario,
    (SELECT MIN(Tab01.dt_fatu) FROM TabPedido AS Tab01 WHERE Tab01.cd_clien=vend_cli.cd_clien
    AND Tab01.cd_vend=vend_cli.cd_vend) AS dt_min,
    (SELECT MAX(Tab01.dt_fatu) FROM TabPedido AS Tab01 WHERE Tab01.cd_clien=vend_cli.cd_clien
    AND Tab01.cd_vend=vend_cli.cd_vend) AS dt_max
    FROM vend_cli) vend_cli
    INNER JOIN cliente ON vend_cli.cd_clien=cliente.cd_clien),

    TabEndereco (cd_clien,tp_end,seq,cep,endereco,bairro,municipio,estado,numero,
    complemento,latitude,longitude,regiao) AS (

    SELECT cd_clien,tp_end,
    ROW_NUMBER()OVER(PARTITION BY cd_clien ORDER BY cd_clien) AS seq,
    cep,endereco,bairro,municipio,estado,numero,
    complemento,latitude,longitude,rot_prdf.descricao AS regiao
    FROM end_cli
    LEFT JOIN rot_cep ON end_cli.cep BETWEEN rot_cep.cep_ini AND rot_cep.cep_fim
    LEFT JOIN rot_prdf ON rot_cep.cd_rot_prdf=rot_prdf.cd_rot_prdf
    ),

    TabContato (cd_clien,seq,tp_tel,ddd,numero) AS (

    SELECT cd_clien,seq,tp_tel,ddd,numero
    FROM tel_cli),

    TabVendedor(cd_vend,nome,nome_resumido,st_vend,cd_equipe,e_mail,categoria) AS (

    SELECT cd_vend,vendedor.nome,
    CASE WHEN CHARINDEX(' ',LTRIM(RTRIM(vendedor.nome)))>0 THEN
    LTRIM(RTRIM(LEFT(vendedor.nome,CHARINDEX(' ',LTRIM(RTRIM(vendedor.nome)))))) + ' '+
    LTRIM(RTRIM(RIGHT(vendedor.nome,CHARINDEX(' ',REVERSE(LTRIM(RTRIM(vendedor.nome)))))))
    ELSE vendedor.nome END AS nome_resumido,
    CASE WHEN vendedor.ativo=1 THEN 'ATIVO' ELSE 'INATIVO' END AS st_vend,
    cd_equipe,usuario.e_mail,categ.descricao AS	categoria
    FROM vendedor
    LEFT JOIN usuario ON vendedor.cd_vend=usuario.cd_usuario
    LEFT JOIN categ ON vendedor.categ=categ.categ),

    TabSupervisor (cd_equipe,equipe,cd_vend_sup,supervisor,email_supervisor,cd_gerente,gerente,email_gerente) AS (

    SELECT supervisor.cd_equipe,supervisor.equipe,cd_vend_sup,
    supervisor,email_supervisor,
    supervisor.cd_gerente,TabVendedor.nome AS gerente,
    TabVendedor.e_mail AS email_gerente
    FROM (
    SELECT equipe.cd_equipe,equipe.descricao AS equipe,cd_vend_sup,
    TabVendedor.nome_resumido AS supervisor,e_mail AS email_supervisor,
    gerencia.descricao AS cd_gerente
    FROM equipe
    INNER JOIN TabVendedor ON equipe.cd_vend_sup=TabVendedor.cd_vend
    INNER JOIN gerencia ON equipe.cd_gerencia=gerencia.cd_gerencia
    )supervisor
    INNER JOIN TabVendedor ON supervisor.cd_gerente=TabVendedor.cd_vend),

    TabPagtoCli (cd_emp,cd_clien,tabela,cond_pgto,prz_medio_max,formpgto,st_cred,tributacao_cliente)  AS (

    SELECT cli_emp.cd_emp,cliente.cd_clien,tab_pre.descricao AS tabela,promocao.descricao AS cond_pgto,prz_medio_max,
    formpgto.descricao AS formpgto,st_cred.descricao AS st_cred,tributacao_cli.descricao AS tributacao_cliente
    FROM cli_emp
    INNER JOIN cliente ON cli_emp.cd_clien=cliente.cd_clien AND cli_emp.cd_emp=cliente.cd_emp
    LEFT JOIN tab_pre ON cli_emp.cd_tabela=tab_pre.cd_tabela
    LEFT JOIN promocao ON cli_emp.seq_prom=promocao.seq_prom
    LEFT JOIN cli_emp_formpgto ON cli_emp.cd_clien=cli_emp_formpgto.cd_clien AND cli_emp.cd_emp=cli_emp_formpgto.cd_emp AND cli_emp_formpgto.principal=1
    LEFT JOIN st_cred ON cliente.st_cred=st_cred.st_cred
    LEFT JOIN formpgto ON cli_emp_formpgto.formpgto=formpgto.formpgto
    LEFT JOIN tributacao_cli ON cliente.seq_trib_cli=tributacao_cli.seq_trib_cli)

    SELECT TabCarteira.cd_clien AS 'ID Cliente',cliente.cgc_cpf AS 'CNPJ',LEN(cliente.cgc_cpf) AS 'Caracter CNPJ',
    cliente.nome AS 'Razão Social',cliente.nome_res AS 'Nome Fantasia',
    CASE WHEN cliente.ativo=1 THEN 'ATIVO' ELSE 'INATIVO' END AS 'Status do Cliente',
    grupocli.descricao AS 'Matriz',cliente.ram_ativ AS 'ID Segmento',
    tabela AS 'Tabela',cond_pgto AS 'Condição de Pagto',prz_medio_max AS 'Prazo',
    formpgto AS 'Forma Pagto',TabPagtoCli.st_cred AS 'Crédito',tributacao_cliente AS 'Tributação',
    cep AS 'CEP',endereco AS 'Endereço',bairro AS 'Bairro',municipio AS 'Cidade',estado AS 'UF',
    TabEndereco.numero AS 'Numero',complemento AS 'Complemento',latitude AS 'Latitude',longitude AS 'Longitude',regiao AS 'Região',
    TabContato.ddd AS 'DDD',TabContato.numero AS 'Contato',
    TabCarteira.cd_vend AS 'ID Vendedor',TabVendedor.nome AS 'Nome',
    TabVendedor.nome_resumido AS 'Nome Resumido',
    TabVendedor.e_mail AS 'E-mail',TabVendedor.categoria AS 'Categoria',
    TabSupervisor.supervisor AS 'Supervisor',TabSupervisor.email_supervisor AS 'Email Sup',
    TabSupervisor.gerente AS 'Gerente',TabSupervisor.email_gerente AS 'Email Gerente',
    CASE WHEN prioritario=1 THEN 'SIM' ELSE 'NÃO' END AS 'Principal',dt_min AS 'Primeira Compra',dt_max AS 'Última Compra',
    contagem AS 'Seq',
    DATEDIFF(DAY,dt_max,dt_atual) AS 'Dias'
    FROM TabCarteira
    INNER JOIN TabVendedor ON TabCarteira.cd_vend=TabVendedor.cd_vend
    INNER JOIN TabSupervisor ON TabVendedor.cd_equipe=TabSupervisor.cd_equipe
    INNER JOIN cliente ON TabCarteira.cd_clien=cliente.cd_clien
    LEFT JOIN grupocli ON cliente.cd_grupocli=grupocli.cd_grupocli
    LEFT JOIN TabPagtoCli ON cliente.cd_clien=TabPagtoCli.cd_clien AND cliente.cd_emp=TabPagtoCli.cd_emp
    LEFT JOIN TabEndereco ON cliente.cd_clien=TabEndereco.cd_clien AND TabEndereco.seq=1
    LEFT JOIN TabContato ON cliente.cd_clien=TabContato.cd_clien AND TabContato.seq=1

    
    """,
    
    'Venda':"""
    

    DECLARE @DIAS SMALLINT,@DTINICIO DATETIME,@DTFIM DATETIME

    SET @DIAS=365

    SET @DTFIM=(SELECT MAX(
    CONVERT(DATETIME,CAST(dt_criacao AS DATE),101))
    FROM evento
    WHERE cd_fila='FATU' /*AND YEAR(dt_criacao)<=YEAR(GETDATE())
    AND MONTH(dt_criacao)<>MONTH(GETDATE())*/
    )

    SET @DTINICIO=DATEADD(DAY,-(DAY(GETDATE())+@DIAS),@DTFIM)

    ;

    WITH TabEvento (seq_evento,nu_ped,cd_fila,cd_emp,cd_clien,dt_criacao ) AS (
    SELECT seq_evento,nu_ped,cd_fila,cd_emp,cd_clien,
    CONVERT(DATETIME,CAST(dt_criacao AS DATE),101)
    FROM evento),

    TabEventoNF (seq_evento,nu_nf,tipo_log,nu_nf_dev,nu_nf_entrada,nota_cliente,seq_nota_cli,nu_nfe) AS (

    SELECT seq_evento,nu_nf,tipo_log,nu_nf_dev,nu_nf_entrada,nota_cliente,seq_nota_cli,
    CASE WHEN nu_nf IS NOT NULL THEN nu_nf ELSE nu_nf_dev END AS nu_nfe
    FROM ev_cnfv),

    TabNFCliente (seq_nota_cli,cd_clien,cd_emp,nu_nf_emp_fat,dt_lanc,dt_emis,dt_receb,
    it_venda_nu_nf,it_venda_seq,cd_prod,unidade,qtde,preco_unit,vl_tot) AS (

    SELECT
    nf_cli.seq_nota_cli,cd_clien,cd_emp,nu_nf_emp_fat,dt_lanc,dt_emis,dt_receb,
    cli_item.it_venda_nu_nf,cli_item.it_venda_seq,
    cli_item.cd_prod,cli_item.unidade,cli_item.qtde,cli_item.preco_unit,cli_item.vl_tot
    FROM nota_cli AS nf_cli
    INNER JOIN nota_cli_item AS cli_item ON nf_cli.seq_nota_cli=cli_item.seq_nota_cli),

    TabNotas (nu_ped,nu_nf,nu_nf_emp_fat,cd_emp) AS (

    SELECT nu_ped,nu_nf,nu_nf_emp_fat,cd_emp
    FROM nota),

    TabCarteira (cd_clien,cd_vend,cd_equipe,prioritario) AS (

    SELECT cd_clien,vend_cli.cd_vend,vendedor.cd_equipe,vend_cli.prioritario 
    FROM vend_cli
    INNER JOIN vendedor ON vend_cli.cd_vend=vendedor.cd_vend),

    TabSituacaoPedido (cd_emp,nu_ped,situacao) AS (

    SELECT DISTINCT cd_emp,nu_ped,
    CASE WHEN situacao IN ('FA','CA','DV') THEN 'FA' ELSE 'AB' END situacao
    FROM it_pedv),

    TabPedidos (cd_emp,nu_ped,cd_clien,cd_vend,cd_equipe,tp_ped,cd_tabela,dt_emissao,dt_faturamento,situacao) AS (

    SELECT DISTINCT
    ped_vda.cd_emp,ped_vda.nu_ped,ped_vda.cd_clien,ped_vda.cd_vend,vendedor.cd_equipe,
    tp_ped,cd_tabela,
    (SELECT MAX(Tab01.dt_criacao) FROM TabEvento AS Tab01 WHERE Tab01.cd_emp=ped_vda.cd_emp 
    AND Tab01.nu_ped=ped_vda.nu_ped AND Tab01.cd_fila='CAPV') AS dt_emissao,
    (SELECT MAX(Tab01.dt_criacao) FROM TabEvento AS Tab01 WHERE Tab01.cd_emp=ped_vda.cd_emp 
    AND Tab01.nu_ped=ped_vda.nu_ped AND Tab01.cd_fila='FATU') AS dt_faturamento,
    TabSituacaoPedido.situacao
    FROM ped_vda
    INNER JOIN TabSituacaoPedido ON ped_vda.cd_emp=TabSituacaoPedido.cd_emp AND ped_vda.nu_ped=TabSituacaoPedido.nu_ped
    LEFT JOIN TabEvento AS Tab01 ON ped_vda.cd_emp=Tab01.cd_emp AND ped_vda.nu_ped=Tab01.nu_ped
    AND ped_vda.cd_clien=Tab01.cd_clien AND Tab01.cd_fila='FATU'
    LEFT JOIN TabEvento AS Tab02 ON ped_vda.cd_emp=Tab02.cd_emp AND ped_vda.nu_ped=Tab02.nu_ped
    AND ped_vda.cd_clien=Tab02.cd_clien AND Tab02.cd_fila='CAPV'
    INNER JOIN vendedor ON ped_vda.cd_vend=vendedor.cd_vend
    ),

    TabPedItens (cd_emp,nu_ped,nu_nf,seq_evento_dev,
    cd_prod,seq,qtde,unid_vda,qtde_unid_vda,vl_unit_vda,vl_venda,cust_cu,peso_brt,peso_liq,
    preco_tabela,desc_apl,perc_com,vl_comis,situacao) AS (

    SELECT it_pedv.cd_emp,nu_ped,nu_nf,seq_evento_dev,
    it_pedv.cd_prod,seq,
    CASE WHEN fator_preco>1 THEN qtde*fator_preco ELSE qtde END AS qtde,
    unid_vda,qtde_unid_vda,vl_unit_vda,qtde_unid_vda*vl_unit_vda AS vl_venda,cust_cue,
    CASE WHEN fator_preco>1 THEN qtde*fator_preco ELSE qtde END*peso_brt AS peso_brt,
    CASE WHEN fator_preco>1 THEN qtde*fator_preco ELSE qtde END*peso_liq AS peso_liq,
    preco_tabela,desc_apl,perc_com,vl_comis,situacao
    FROM it_pedv
    INNER JOIN produto ON it_pedv.cd_prod=produto.cd_prod),

    /*Tabela de Devoluções e cancelamentos*/
    TabEventoCNFV (seq_evento,tipo_log,nu_nfe,nu_ped,cd_clien,cd_vend,cd_equipe,tp_ped,cd_tabela,cd_emp,dt_emissao,dt_faturamento,it_venda_seq,
    cd_prod,qtde,unid_vda,qtde_unid_vda,vl_unit_vda,vl_venda,cust_cu,peso_brt,peso_liq,
    preco_tabela,desc_apl,vl_desc,perc_com,vl_comis,situacao) AS (

    SELECT TabEventoNF.seq_evento,tipo_log,nu_nfe,
    TabEvento.nu_ped,TabEvento.cd_clien,TabPedidos.cd_vend,vendedor.cd_equipe,
    TabPedidos.tp_ped,TabPedidos.cd_tabela,
    TabEvento.cd_emp,TabPedidos.dt_emissao,TabEvento.dt_criacao AS dt_faturamento,
    TabNFCliente.it_venda_seq,
    TabNFCliente.cd_prod,
    TabNFCliente.qtde,TabNFCliente.unidade,TabNFCliente.qtde,TabNFCliente.preco_unit,
    TabNFCliente.vl_tot,
    CONVERT(DECIMAL(15,2),TabPedItens.cust_cu/TabPedItens.qtde)*TabNFCliente.qtde AS cust_cu,
    CONVERT(DECIMAL(15,4),TabNFCliente.qtde*produto.peso_brt) AS peso_brt,
    CONVERT(DECIMAL(15,4),TabNFCliente.qtde*produto.peso_liq) AS peso_liq,
    preco_tabela,desc_apl,CONVERT(DECIMAL(15,4),ROUND(preco_tabela*desc_apl,2)) AS vl_desc,
    perc_com,vl_comis,
    CASE WHEN TabNFCliente.it_venda_seq IS NOT NULL THEN tipo_log ELSE TabPedItens.situacao END AS situacao
    FROM TabEventoNF
    INNER JOIN TabEvento ON TabEventoNF.seq_evento=TabEvento.seq_evento
    INNER JOIN TabPedidos ON TabEvento.cd_emp=TabPedidos.cd_emp AND TabEvento.nu_ped=TabPedidos.nu_ped
    INNER JOIN vendedor ON TabPedidos.cd_vend=vendedor.cd_vend
    LEFT JOIN TabNFCliente ON TabEventoNF.seq_nota_cli=TabNFCliente.seq_nota_cli
    LEFT JOIN TabPedItens ON TabEventoNF.nu_nfe=TabPedItens.nu_nf AND
    TabNFCliente.it_venda_seq=TabPedItens.seq AND TabEvento.cd_emp=TabPedItens.cd_emp
    INNER JOIN produto ON TabNFCliente.cd_prod=produto.cd_prod
    WHERE TabNFCliente.it_venda_seq IS NOT NULL
    UNION ALL
    SELECT TabEventoNF.seq_evento,tipo_log,nu_nfe,
    TabEvento.nu_ped,TabEvento.cd_clien,TabPedidos.cd_vend,vendedor.cd_equipe,
    TabPedidos.tp_ped,TabPedidos.cd_tabela,
    TabEvento.cd_emp,TabPedidos.dt_emissao,TabEvento.dt_criacao AS dt_faturamento,
    TabPedItens.seq,
    TabPedItens.cd_prod,
    TabPedItens.qtde,TabPedItens.unid_vda,TabPedItens.qtde_unid_vda,TabPedItens.vl_unit_vda,
    TabPedItens.vl_venda,
    TabPedItens.cust_cu,TabPedItens.peso_brt,TabPedItens.peso_liq,
    preco_tabela,desc_apl,CONVERT(DECIMAL(15,4),ROUND(preco_tabela*desc_apl,2)) AS vl_desc,
    perc_com,vl_comis,TabPedItens.situacao
    FROM TabEventoNF
    INNER JOIN TabEvento ON TabEventoNF.seq_evento=TabEvento.seq_evento
    INNER JOIN TabPedidos ON TabEvento.cd_emp=TabPedidos.cd_emp AND TabEvento.nu_ped=TabPedidos.nu_ped
    INNER JOIN vendedor ON TabPedidos.cd_vend=vendedor.cd_vend
    LEFT JOIN TabNFCliente ON TabEventoNF.seq_nota_cli=TabNFCliente.seq_nota_cli
    LEFT JOIN TabPedItens ON TabEventoNF.nu_nfe=TabPedItens.nu_nf AND
    TabEvento.cd_emp=TabPedItens.cd_emp
    WHERE TabNFCliente.it_venda_seq IS NULL),

    TabSituacao (tipo_log,desc_situacao) AS (

    SELECT tipo_log,
    CASE WHEN tipo_log='CA' THEN 'CANCELADO' WHEN tipo_log='DV' THEN 'DEVOLUÇÃO'
    WHEN tipo_log='DP' THEN 'DEVOLUÇÃO PARCIAL' WHEN tipo_log='FA' THEN 'FATURADO'
    ELSE 'EM ABERTO' END desc_situacao
    FROM (
    SELECT DISTINCT tipo_log 
    FROM TabEventoCNFV
    UNION ALL
    SELECT DISTINCT situacao FROM TabPedidos)C),

    TabTipoPedido (tp_ped,descricao,estat_com,bonificacao,tipo_pedido) AS (

    SELECT tp_ped,descricao,estat_com,bonificacao,
    CASE WHEN estat_com=1 THEN 'VENDAS' WHEN bonificacao=1
    THEN LEFT(descricao,CHARINDEX(' ',LTRIM(RTRIM(descricao)))) ELSE 'OUTROS' END tipo_pedido
    FROM tp_ped),

    TabConsolidado (cd_emp,nu_ped,cd_clien,cd_vend,cd_equipe,tp_ped,cd_tabela,dt_emissao,dt_faturamento,situacao,nu_nf,
    cd_prod,seq,qtde,unid_vda,qtde_unid_vda,vl_unit_vda,vl_venda,cust_cu,peso_brt,peso_liq,
    preco_tabela,desc_apl,perc_com,vl_comis) AS (

    SELECT TabPedidos.cd_emp,TabPedidos.nu_ped,cd_clien,cd_vend,TabPedidos.cd_equipe,tp_ped,cd_tabela,dt_emissao,dt_faturamento,TabPedidos.situacao 
    ,nu_nf,
    cd_prod,seq,qtde,unid_vda,qtde_unid_vda,vl_unit_vda,vl_venda,cust_cu,peso_brt,peso_liq,
    preco_tabela,desc_apl,perc_com,vl_comis
    FROM TabPedidos
    INNER JOIN TabPedItens ON TabPedidos.cd_emp=TabPedItens.cd_emp AND TabPedidos.nu_ped=TabPedItens.nu_ped
    UNION ALL
    /*A query abaixo representa as devoluções e os cancelamentos*/
    SELECT cd_emp,nu_ped,cd_clien,cd_vend,cd_equipe,tp_ped,cd_tabela,dt_emissao,dt_faturamento,tipo_log,nu_nfe,
    cd_prod,it_venda_seq,qtde*-1,unid_vda,qtde_unid_vda*-1,vl_unit_vda,vl_venda*-1,cust_cu*-1,
    peso_brt*-1,peso_liq*-1,preco_tabela,desc_apl*-1,perc_com,vl_comis*-1
    FROM TabEventoCNFV
    WHERE TabEventoCNFV.situacao<>'FA'
    ),

    TabVendCad (cd_emp,nu_ped,cd_clien,
    cd_vend,cad_vend,cd_equipe,tp_ped,cd_tabela,dt_emissao,dt_faturamento,situacao,nu_nf,
    cd_prod,seq,qtde,unid_vda,qtde_unid_vda,vl_unit_vda,vl_venda,cust_cu,peso_brt,peso_liq,
    preco_tabela,desc_apl,perc_com,vl_comis) AS (

    SELECT DISTINCT cd_emp,nu_ped,TabConsolidado.cd_clien,
    TabConsolidado.cd_vend,ISNULL(ISNULL(Tab01.cd_vend,Tab02.cd_vend),Tab03.cd_vend) AS cad_vend,
    TabConsolidado.cd_equipe,tp_ped,cd_tabela,dt_emissao,dt_faturamento,situacao,nu_nf,
    cd_prod,seq,qtde,unid_vda,qtde_unid_vda,vl_unit_vda,vl_venda,cust_cu,peso_brt,peso_liq,
    preco_tabela,desc_apl,perc_com,vl_comis
    FROM TabConsolidado
    LEFT JOIN TabCarteira AS Tab01 ON TabConsolidado.cd_clien=Tab01.cd_clien 
    AND TabConsolidado.cd_vend=Tab01.cd_vend
    LEFT JOIN TabCarteira AS Tab02 ON TabConsolidado.cd_clien=Tab02.cd_clien 
    AND TabConsolidado.cd_equipe=Tab02.cd_equipe AND Tab02.prioritario=1
    LEFT JOIN TabCarteira AS Tab03 ON TabConsolidado.cd_clien=Tab03.cd_clien
    AND Tab03.prioritario=1),

    TabPedVendedor (cd_emp,nu_ped,cd_clien,
    tipo_vend,id_vend,cd_equipe,tp_ped,cd_tabela,dt_emissao,dt_faturamento,situacao,nu_nf,
    cd_prod,seq,qtde,unid_vda,qtde_unid_vda,vl_unit_vda,vl_venda,cust_cu,peso_brt,peso_liq,
    preco_tabela,desc_apl,perc_com,vl_comis) AS (

    SELECT cd_emp,nu_ped,cd_clien,
    tipo_vend,id_vend,cd_equipe,tp_ped,cd_tabela,dt_emissao,dt_faturamento,situacao,nu_nf,
    cd_prod,seq,qtde,unid_vda,qtde_unid_vda,vl_unit_vda,vl_venda,cust_cu,peso_brt,peso_liq,
    preco_tabela,desc_apl,perc_com,vl_comis FROM (
    SELECT cd_emp,nu_ped,cd_clien,
    cd_vend,cad_vend,cd_equipe,tp_ped,cd_tabela,dt_emissao,dt_faturamento,situacao,nu_nf,
    cd_prod,seq,qtde,unid_vda,qtde_unid_vda,vl_unit_vda,vl_venda,cust_cu,peso_brt,peso_liq,
    preco_tabela,desc_apl,perc_com,vl_comis
    FROM TabVendCad)C
    UNPIVOT(id_vend FOR tipo_vend IN (cd_vend,cad_vend))L),

    TabPedDTCriacao (cd_emp,nu_ped,cd_clien,
    tipo_vend,id_vend,cd_equipe,tp_ped,cd_tabela,tipo_data,dt_criacao,situacao,nu_nf,
    cd_prod,seq,qtde,unid_vda,qtde_unid_vda,vl_unit_vda,vl_venda,cust_cu,peso_brt,peso_liq,
    preco_tabela,desc_apl,perc_com,vl_comis) AS (

    SELECT cd_emp,nu_ped,cd_clien,
    tipo_vend,id_vend,cd_equipe,tp_ped,cd_tabela,tipo_data,dt_criacao,situacao,nu_nf,
    cd_prod,seq,qtde,unid_vda,qtde_unid_vda,vl_unit_vda,vl_venda,cust_cu,peso_brt,peso_liq,
    preco_tabela,desc_apl,perc_com,vl_comis FROM (
    SELECT cd_emp,nu_ped,cd_clien,
    tipo_vend,id_vend,cd_equipe,tp_ped,cd_tabela,dt_emissao,dt_faturamento,situacao,nu_nf,
    cd_prod,seq,qtde,unid_vda,qtde_unid_vda,vl_unit_vda,vl_venda,cust_cu,peso_brt,peso_liq,
    preco_tabela,desc_apl,perc_com,vl_comis 
    FROM TabPedVendedor)C
    UNPIVOT(dt_criacao FOR tipo_data IN (dt_emissao,dt_faturamento))L),

    TabPedPeso (cd_emp,nu_ped,cd_clien,
    tipo_vend,id_vend,cd_equipe,tp_ped,cd_tabela,tipo_data,dt_criacao,situacao,nu_nf,
    cd_prod,seq,qtde,unid_vda,qtde_unid_vda,vl_unit_vda,vl_venda,cust_cu,tipo_peso,peso,
    preco_tabela,desc_apl,perc_com,vl_comis) AS (

    SELECT cd_emp,nu_ped,cd_clien,
    tipo_vend,id_vend,cd_equipe,tp_ped,cd_tabela,tipo_data,dt_criacao,situacao,nu_nf,
    cd_prod,seq,qtde,unid_vda,qtde_unid_vda,vl_unit_vda,vl_venda,cust_cu,tipo_peso,peso,
    preco_tabela,desc_apl,perc_com,vl_comis FROM (
    SELECT cd_emp,nu_ped,cd_clien,
    tipo_vend,id_vend,cd_equipe,tp_ped,cd_tabela,tipo_data,dt_criacao,situacao,nu_nf,
    cd_prod,seq,qtde,unid_vda,qtde_unid_vda,vl_unit_vda,vl_venda,cust_cu,peso_brt,peso_liq,
    preco_tabela,desc_apl,perc_com,vl_comis 
    FROM TabPedDTCriacao)C
    UNPIVOT(peso FOR tipo_peso IN (peso_brt,peso_liq))L)

    SELECT * FROM (
    SELECT TabPedPeso.cd_emp AS 'ID Empresa',TabPedPeso.nu_ped AS 'Numero do Pedido',TabNotas.nu_nf_emp_fat AS 'NFe',cd_clien AS 'ID Cliente',
    UPPER(tipo_vend) AS 'Tipo de Vendedor',id_vend AS 'ID Vendedor',TabTipoPedido.descricao AS 'Tipo de Pedido',
    TabTipoPedido.tipo_pedido AS 'Tipo de Operação',tab_pre.descricao AS 'Tabela de Preço',
    UPPER(tipo_data) AS 'Tipo de Data',dt_criacao AS 'Data de Criação',TabSituacao.desc_situacao AS 'Status do Pedido',TabPedPeso.nu_nf AS 'ID NFe',
    cd_prod AS 'SKU',seq AS 'Seq',qtde AS 'Qtde',unid_vda AS 'Unid. VDA',qtde_unid_vda AS 'Qtde VDA',
    vl_unit_vda AS 'Valor VDA',vl_venda AS 'Total Geral',cust_cu AS 'Custo da Útima Entrada',UPPER(tipo_peso) AS 'Tipo de Peso',peso AS 'Peso',
    preco_tabela AS 'Preço da Tabela',desc_apl AS '% Desconto',perc_com AS '% Comissão',vl_comis AS 'Comissão R$'
    FROM TabPedPeso
    INNER JOIN TabSituacao ON TabPedPeso.situacao=TabSituacao.tipo_log
    INNER JOIN TabTipoPedido ON TabPedPeso.tp_ped=TabTipoPedido.tp_ped
    INNER JOIN tab_pre ON TabPedPeso.cd_tabela=tab_pre.cd_tabela
    INNER JOIN TabNotas ON TabPedPeso.nu_nf=TabNotas.nu_nf AND TabPedPeso.nu_ped=TabNotas.nu_ped AND TabPedPeso.cd_emp=TabNotas.cd_emp
    UNION ALL
    SELECT TabPedPeso.cd_emp AS 'ID Empresa',TabPedPeso.nu_ped AS 'Numero do Pedido',TabNotas.nu_nf_emp_fat AS 'NFe',cd_clien AS 'ID Cliente',
    UPPER(tipo_vend) AS 'Tipo de Vendedor',id_vend AS 'ID Vendedor',TabTipoPedido.descricao AS 'Tipo de Pedido',
    TabTipoPedido.tipo_pedido AS 'Tipo de Operação',tab_pre.descricao AS 'Tabela de Preço',
    UPPER(tipo_data) AS 'Tipo de Data',dt_criacao AS 'Data de Criação',TabSituacao.desc_situacao AS 'Status do Pedido',TabPedPeso.nu_nf AS 'ID NFe',
    cd_prod AS 'SKU',seq AS 'Seq',qtde AS 'Qtde',unid_vda AS 'Unid. VDA',qtde_unid_vda AS 'Qtde VDA',
    vl_unit_vda AS 'Valor VDA',vl_venda AS 'Total Geral',cust_cu AS 'Custo da Útima Entrada',UPPER(tipo_peso) AS 'Tipo de Peso',peso AS 'Peso',
    preco_tabela AS 'Preço da Tabela',desc_apl AS '% Desconto',perc_com AS '% Comissão',vl_comis AS 'Comissão R$'
    FROM TabPedPeso
    INNER JOIN TabSituacao ON TabPedPeso.situacao=TabSituacao.tipo_log
    INNER JOIN TabTipoPedido ON TabPedPeso.tp_ped=TabTipoPedido.tp_ped
    INNER JOIN tab_pre ON TabPedPeso.cd_tabela=tab_pre.cd_tabela
    LEFT JOIN TabNotas ON TabPedPeso.nu_nf=TabNotas.nu_nf AND TabPedPeso.nu_ped=TabNotas.nu_ped AND TabPedPeso.cd_emp=TabNotas.cd_emp
    WHERE TabPedPeso.situacao='AB')C
    WHERE C.[Data de Criação] BETWEEN @DTINICIO AND @DTFIM
    ORDER BY C.[Numero do Pedido]
    
    
    """,
    
    'Supervisor':"""
    
    
    WITH TabVendedor(cd_vend,nome,nome_resumido,st_vend,cd_equipe,e_mail,categoria) AS (

    SELECT cd_vend,vendedor.nome,
    CASE WHEN CHARINDEX(' ',LTRIM(RTRIM(vendedor.nome)))>0 THEN
    LTRIM(RTRIM(LEFT(vendedor.nome,CHARINDEX(' ',LTRIM(RTRIM(vendedor.nome)))))) + ' '+
    LTRIM(RTRIM(RIGHT(vendedor.nome,CHARINDEX(' ',REVERSE(LTRIM(RTRIM(vendedor.nome)))))))
    ELSE vendedor.nome END AS nome_resumido,
    CASE WHEN vendedor.ativo=1 THEN 'ATIVO' ELSE 'INATIVO' END AS st_vend,
    cd_equipe,usuario.e_mail,categ.descricao AS	categoria
    FROM vendedor
    LEFT JOIN usuario ON vendedor.cd_vend=usuario.cd_usuario
    LEFT JOIN categ ON vendedor.categ=categ.categ),

    TabSupervisor (cd_equipe,equipe,cd_vend_sup,supervisor,email_supervisor,cd_gerente,gerente,email_gerente) AS (

    SELECT supervisor.cd_equipe,supervisor.equipe,cd_vend_sup,
    supervisor,email_supervisor,
    supervisor.cd_gerente,TabVendedor.nome AS gerente,
    TabVendedor.e_mail AS email_gerente
    FROM (
    SELECT equipe.cd_equipe,equipe.descricao AS equipe,cd_vend_sup,
    TabVendedor.nome_resumido AS supervisor,e_mail AS email_supervisor,
    gerencia.descricao AS cd_gerente
    FROM equipe
    INNER JOIN TabVendedor ON equipe.cd_vend_sup=TabVendedor.cd_vend
    INNER JOIN gerencia ON equipe.cd_gerencia=gerencia.cd_gerencia
    )supervisor
    INNER JOIN TabVendedor ON supervisor.cd_gerente=TabVendedor.cd_vend)

    SELECT cd_equipe AS 'ID Equipe',equipe AS 'Equipe',cd_vend_sup AS 'ID Sup',
    supervisor AS 'Supervisor',email_supervisor AS 'Email Sup',
    cd_gerente AS 'ID Gerente',gerente AS 'Gerente',email_gerente AS 'Email Gerente'
    FROM TabSupervisor 
    
    
    """,
    
    'Vendedor':"""
    
    SELECT V.CD_VEND AS 'ID Vendedor',V.NOME AS 'Vendedor',
    CASE WHEN 
    CHARINDEX(' ',LTRIM(RTRIM(V.NOME)))=0 THEN V.NOME ELSE
    LTRIM(RTRIM(LEFT(V.NOME,CHARINDEX(' ',LTRIM(RTRIM(V.NOME))))))+ ' '+
    LTRIM(RTRIM(RIGHT(V.NOME,CHARINDEX(' ',REVERSE(LTRIM(RTRIM(V.NOME))))))) 
    END AS 'Nome Resumido',V.CD_EQUIPE AS 'ID Equipe',
    U.E_MAIL AS 'E-mail',categ.descricao AS 'Categoria'
    FROM VENDEDOR AS V
    LEFT JOIN USUARIO AS U ON V.CD_VEND=U.CD_USUARIO
    LEFT JOIN categ ON V.CATEG=categ.categ
    WHERE V.ATIVO=1    
    
    
    """,
    
    'Calendario':"""
    
    
    DECLARE @DTInicial AS DATETIME,@DTFinal AS DATETIME

    SET @DTInicial='2018-01-01'
    SET @DTFinal= CONCAT(YEAR(GETDATE())+1,'-01-','01')

    ;WITH Calendario (Datas) AS(

    SELECT @DTInicial
    UNION ALL
    SELECT Datas+1
    FROM Calendario WHERE  Datas+1<@DTFinal
    )

    SELECT CONVERT(DATETIME,CAST(Datas AS DATE),101) AS 'Data',YEAR(Datas) AS 'Ano',MONTH(Datas) AS 'Cód. Mês',
    CHOOSE(MONTH(Datas),'JANEIRO','FEVEREIRO','MARÇO','ABRIL','MAIO','JUNHO','JULHO','AGOSTO','SETEMBRO','OUTUBRO','NOVEMBRO','DEZEMBRO') AS 'Mês',
    CHOOSE(MONTH(Datas),'JAN','FEV','MAR','ABR','MAI','JUN','JUL','AGO','SET','OUT','NOV','DEZ') AS 'Mês Resumido',
    DAY(Datas) AS 'Dia',CONVERT(VARCHAR(7),Datas,120) AS 'Mês Meta',
    DATEPART(DW,Datas) AS 'Cód. Semana',CHOOSE(DATEPART(DW,Datas),'DOM','SEG','TER','QUAR','QUI','SEX','SÁB') AS 'Semana',
    CASE WHEN DATEPART(DW,Datas) IN (7,1) THEN 0 ELSE 1 END AS 'Dias Úteis',
    CASE WHEN MONTH(Datas)<=3 THEN '1º TRIM' WHEN MONTH(Datas)<=6 THEN '2º TRIM' WHEN MONTH(Datas)<=9 THEN '3º TRIM' WHEN MONTH(Datas)<=12 THEN '4º TRIM' END AS 'Trimestre Ano',
    CASE WHEN MONTH(Datas)<=6 THEN '1º SEM' ELSE '2º SEM' END AS 'Semestre Ano',DATEPART(WEEK,Datas) AS 'Semana Ano',
    CONVERT(VARCHAR,DAY(Datas))+'/'+CONVERT(VARCHAR,MONTH(Datas)) AS 'Mês Base'
    FROM Calendario OPTION(MAXRECURSION 10000)    
    
    
    """,
    
    'Feriado':"""
        
    DECLARE @DateInicial AS DATETIME,@DateFinal AS DATETIME

    SET @DateInicial='2018-01-01'
    SET @DateFinal=CONCAT(YEAR(GETDATE())+1,'-01-01')

    ;WITH TabDatas(dt_calend) AS (
    SELECT @DateInicial
    UNION ALL
    SELECT dt_calend+1 FROM TabDatas WHERE dt_calend+1<@DateFinal),

    TabCalendario (dt_calend,mes_base) AS(

    SELECT CONVERT(DATETIME,CAST(dt_calend AS DATE),101),CONVERT(VARCHAR,DAY(dt_calend))+'/'+CONVERT(VARCHAR,MONTH(dt_calend)) AS mes_base
    FROM TabDatas 
    ),

    TabFeriado (dt_calend,mes_base,dias_feriado) AS (

    SELECT dt_calend,mes_base,
    CASE WHEN mes_base IN ('1/1','10/4','21/4','1/5','11/6','7/9','12/10','2/11','15/11','25/12','8/4') THEN 0 ELSE 1 END 
    FROM TabCalendario)

    SELECT DISTINCT mes_base AS 'Mês Base'
    FROM TabFeriado
    WHERE dias_feriado=0
    ORDER BY 1
    OPTION(MAXRECURSION 10000)    
    
    """,
    
    'Segmento':"""
    
    SELECT ram_ativ AS 'ID Segmento',descricao AS 'Segmento',
    CASE WHEN descricao IN (

    '10 OU + CHK',
     'EMPÓRIO',
     'MERCEARIA',
     'BOMBONIERE',
     'AÇOUGUE',
     'ACADEMIA',
     'LOJA DE CONVENIÊNCIA',
     'FORNECEDOR',
     'SACOLÃO',
     'FARMACIA',
     'DISTRIBUIDOR',
     'BANCA DE JORNAL',
     '1 A 4 CHK',
     '5 A 9 CHK',
     'CANAL ESPECIALIZADO',
     'ATACAREJO'

    ) THEN 'AS' ELSE 'FS' END AS 'Canal'
    --CASE WHEN ativo=1 THEN 'ATIVO' ELSE 'INATIVO' END AS 'Status'
    FROM ram_ativ
    WHERE ativo=1    
    
    
    """,
    
    'Meta':"""
    
    SET NOCOUNT ON;

    Declare @Coluna_Pivot AS NVARCHAR(MAX),@Comando AS NVARCHAR(MAX)

    SET @Coluna_Pivot=STUFF((SELECT DISTINCT ', ' + cd_tp_prev FROM prev_vda FOR XML PATH('')),1,1,'')

    SET @Comando='
    SELECT * FROM (
    SELECT prev_vda.cd_prev_vda,mes_ref,CONVERT(VARCHAR(7),mes_ref,120) mes_meta,
    cd_tp_prev,it_prev_vda.cd_vend,
    valor
    FROM prev_vda
    INNER JOIN it_prev_vda ON prev_vda.cd_prev_vda=it_prev_vda.cd_prev_vda AND prev_vda.cd_tp_prev=it_prev_vda.cd_tp_prev_det)linha
    PIVOT(SUM(valor) FOR cd_tp_prev IN ('+@Coluna_Pivot+'))coluna'

    EXECUTE(@Comando)    
    
    
    """
    
}

In [3]:
tabela_dict=dict()

for tabela,query in querys.items():
    
    tabela_dict[tabela]=pd.read_sql(query,conectando)
    
    pass

In [4]:
tabela_dict['Segmento'].columns

Index(['ID Segmento', 'Segmento', 'Canal'], dtype='object')

In [5]:
tabela_dict['Carteira'].columns

Index(['ID Cliente', 'CNPJ', 'Caracter CNPJ', 'Razão Social', 'Nome Fantasia',
       'Status do Cliente', 'Matriz', 'ID Segmento', 'Tabela',
       'Condição de Pagto', 'Prazo', 'Forma Pagto', 'Crédito', 'Tributação',
       'CEP', 'Endereço', 'Bairro', 'Cidade', 'UF', 'Numero', 'Complemento',
       'Latitude', 'Longitude', 'Região', 'DDD', 'Contato', 'ID Vendedor',
       'Nome', 'Nome Resumido', 'E-mail', 'Categoria', 'Supervisor',
       'Email Sup', 'Gerente', 'Email Gerente', 'Principal', 'Primeira Compra',
       'Última Compra', 'Seq', 'Dias'],
      dtype='object')

In [6]:
#Carteira do vendedor clientes acima de 90 dias até 365

tabela_dict['Carteira']=tabela_dict['Carteira'].merge(tabela_dict['Segmento'],on='ID Segmento',how='inner')[['ID Cliente', 'CNPJ', 'Caracter CNPJ', 'Razão Social', 'Nome Fantasia',
       'Status do Cliente', 'Matriz', 'Segmento', 'Canal', 'Tabela',
       'Condição de Pagto', 'Prazo', 'Forma Pagto', 'Crédito', 'Tributação',
       'CEP', 'Endereço', 'Bairro', 'Cidade', 'UF', 'Numero', 'Complemento',
       'Latitude', 'Longitude', 'Região', 'DDD', 'Contato', 'ID Vendedor',
       'Nome', 'Nome Resumido', 'E-mail', 'Categoria', 'Supervisor',
       'Email Sup', 'Gerente', 'Email Gerente', 'Principal', 'Primeira Compra',
       'Última Compra', 'Seq', 'Dias']]

In [7]:
tabela_dict['Carteira']=tabela_dict['Carteira'].loc[(tabela_dict['Carteira']['Dias']<=365)&(tabela_dict['Carteira']['Categoria']=='CLT')&(tabela_dict['Carteira']['Status do Cliente']=='ATIVO')&(~tabela_dict['Carteira']['E-mail'].isnull())]

In [8]:
tabela_dict['Venda'].columns

Index(['ID Empresa', 'Numero do Pedido', 'NFe', 'ID Cliente',
       'Tipo de Vendedor', 'ID Vendedor', 'Tipo de Pedido', 'Tipo de Operação',
       'Tabela de Preço', 'Tipo de Data', 'Data de Criação',
       'Status do Pedido', 'ID NFe', 'SKU', 'Seq', 'Qtde', 'Unid. VDA',
       'Qtde VDA', 'Valor VDA', 'Total Geral', 'Custo da Útima Entrada',
       'Tipo de Peso', 'Peso', 'Preço da Tabela', '% Desconto', '% Comissão',
       'Comissão R$'],
      dtype='object')

In [9]:
vendas_df=tabela_dict['Venda'].loc[(tabela_dict['Venda']['Tipo de Vendedor']=='CD_VEND')&(tabela_dict['Venda']['Tipo de Data']=='DT_FATURAMENTO')&(tabela_dict['Venda']['Tipo de Peso']=='PESO_BRT')&(tabela_dict['Venda']['Tipo de Operação']=='VENDAS')]

In [10]:
from datetime import datetime,timedelta

data_max=vendas_df['Data de Criação'].max()

data_inicial=data_max-timedelta(days=365)

data_atual=datetime.now().date()

In [11]:
grpcliente_df=vendas_df[['ID Cliente','ID Vendedor','Total Geral','Numero do Pedido']].loc[(vendas_df['Data de Criação'].between(data_inicial,data_max))].groupby(['ID Cliente','ID Vendedor'],as_index=False).agg({'Total Geral':'sum','Numero do Pedido':'count'})

In [12]:
tabela_dict['Carteira']['Total Geral']=tabela_dict['Carteira'].apply(lambda info: grpcliente_df['Total Geral'].loc[(grpcliente_df['ID Cliente']==info['ID Cliente'])&((grpcliente_df['ID Vendedor']==info['ID Vendedor']))].sum(),axis=1)

tabela_dict['Carteira']['Pedido']=tabela_dict['Carteira'].apply(lambda info: grpcliente_df['Numero do Pedido'].loc[(grpcliente_df['ID Cliente']==info['ID Cliente'])&(grpcliente_df['ID Vendedor']==info['ID Vendedor'])].sum(),axis=1)

In [13]:
tabela_dict['Carteira'].loc[(tabela_dict['Carteira']['Última Compra'].dt.year==data_atual.year)&(tabela_dict['Carteira']['Última Compra'].dt.month==data_atual.month),'Comprou']='SIM'

tabela_dict['Carteira'].loc[(tabela_dict['Carteira']['Comprou'].isnull()),'Comprou']='NÃO'

tabela_dict['Carteira']

,ID Cliente,CNPJ,Caracter CNPJ,Razão Social,Nome Fantasia,Status do Cliente,Matriz,Segmento,Canal,Tabela,Condição de Pagto,Prazo,Forma Pagto,Crédito,Tributação,CEP,Endereço,Bairro,Cidade,UF,Numero,Complemento,Latitude,Longitude,Região,DDD,Contato,ID Vendedor,Nome,Nome Resumido,E-mail,Categoria,Supervisor,Email Sup,Gerente,Email Gerente,Principal,Primeira Compra,Última Compra,Seq,Dias,Total Geral,Pedido,Comprou
2,2145,02357205000154,14,CECILIA TSUYACO ARAKI SILVA,CECILIA TSUYACO ARAK,ATIVO,None,DISTRIBUIDOR,AS,DISTRIBUIDORES,BOLETO BANCARIO 21 DIAS,21.0,None,NORMAL,PESSOA JURIDICA - CONTRI NORMAL,5314000.0,AVENIDA DR. GASTAO VIDIGAL,VILA LEOPOLDINA,SÃO PAULO,SP,1946,None,NaN,NaN,JAGUARE/LEOPOLDINA,11,38326362.0,LOJA,LOJA,LOJA,,CLT,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,SIM,2018-03-01,2022-06-01,1,1,197152.13,1554,SIM
3,3626,00537053000146,14,PONTUAL COMERCIAL AGRICOLA LTDA,PONTUAL COMERCIAL AG,ATIVO,None,DISTRIBUIDOR,AS,TABELA DE PREÇO,BOLETO BANCARIO 28 DIAS,28.0,BANCO,NORMAL,PESSOA JURIDICA - CONTRI NORMAL,5316900.0,AVENIDA DR GASTAO VIDIGAL MLP M269,VILA LEOPOLDINA,SÃO PAULO,SP,1946,None,NaN,NaN,JAGUARE/LEOPOLDINA,11,36438330.0,LOJA,LOJA,LOJA,,CLT,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,SIM,2018-03-27,2022-03-17,1,77,757.36,3,NÃO
5,4837,08182013000130,14,MARIO PEDRO FELICIANO - HORTIFRUTI - EPP,ELO FRUTI,ATIVO,None,DISTRIBUIDOR,AS,TABELA DE PREÇO,BOLETO BANCARIO 14 DIAS,14.0,BANCO,NORMAL,PESSOA JURIDICA - CONTRI NORMAL,5316900.0,AVENIDA DOUTOR GASTAO VIDIGAL,VILA LEOPOLDINA,SÃO PAULO,SP,1946,None,NaN,NaN,JAGUARE/LEOPOLDINA,11,36439441.0,LOJA,LOJA,LOJA,,CLT,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,SIM,2018-03-08,2022-05-05,1,28,6906.44,20,NÃO
6,5398,62891254000102,14,AGRO COMERCIAL BONFIM PAULISTA LTDA EPP,AGRO BONFIM,ATIVO,None,DISTRIBUIDOR,AS,DISTRIBUIDORES,A VISTA,1.0,DINHEIRO,NORMAL,PESSOA JURIDICA - CONTRI NORMAL,5316900.0,AVENIDA DOUTOR GASTAO VIDIGAL,VILA LEOPOLDINA,SÃO PAULO,SP,1946,None,NaN,NaN,JAGUARE/LEOPOLDINA,11,36444169.0,LOJA,LOJA,LOJA,,CLT,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,SIM,2018-03-01,2022-06-01,1,1,290194.20,3154,SIM
7,5890,03563866000107,14,ILKA NATÁLIA RODRIGUES HORTIFRUTI,HORT CLEAN,ATIVO,None,DISTRIBUIDOR,AS,DISTRIBUIDORES,A VISTA,1.0,DINHEIRO,NORMAL,PESSOA JURIDICA - CONTRI SIMPLES,5314900.0,DR.GASTAO VIDIGAL,VILA LEOPOLDINA,SÃO PAULO,SP,1946,None,NaN,NaN,JAGUARE/LEOPOLDINA,11,36438814.0,LOJA,LOJA,LOJA,,CLT,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,SIM,2018-03-01,2022-06-01,1,1,2355.28,30,SIM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12858,26963,12508456000146,14,G2F COMERCIO DE ALIMENTOS LTDA,SUBWAY SHOP LARGO 13,ATIVO,None,FAST FOOD/REDES,FS,TABELA DE PREÇO,A VISTA,1.0,DINHEIRO,NORMAL,PESSOA JURIDICA - CONTRI SIMPLES,4752005.0,"RUA AMADOR BUENO, 219",SANTO AMARO,SÃO PAULO,SP,219,,NaN,NaN,SANTO AMARO,11,992550240.0,JULIODEL,JULIO C DELFINO,JULIO DELFINO,,CLT,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,SIM,2021-12-26,2021-12-26,1,158,0.00,0,NÃO
12859,26964,32024186000105,14,GRANSOMMER2 COMERCIO DE ALIMENTOS LTDA,SUBWAY TATUAPE,ATIVO,None,FAST FOOD/REDES,FS,TABELA DE PREÇO,A VISTA,1.0,DINHEIRO,NORMAL,PESSOA JURIDICA - CONTRI SIMPLES,3066030.0,"RUA GONÇALVES CRESPO, 78 - LOJA 415",TATUAPÉ,SÃO PAULO,SP,78,LOJA 415,NaN,NaN,BRAS,11,20965373.0,JULIODEL,JULIO C DELFINO,JULIO DELFINO,,CLT,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,SIM,2022-01-06,2022-01-06,1,147,1199.70,1,NÃO
12861,26981,37141100000148,14,CANT

In [14]:
tabela_dict['Vendedor'].columns

Index(['ID Vendedor', 'Vendedor', 'Nome Resumido', 'ID Equipe', 'E-mail',
       'Categoria'],
      dtype='object')

In [15]:
tabela_dict['Supervisor'].columns

Index(['ID Equipe', 'Equipe', 'ID Sup', 'Supervisor', 'Email Sup',
       'ID Gerente', 'Gerente', 'Email Gerente'],
      dtype='object')

In [16]:
tabela_dict['Vendedor']=tabela_dict['Vendedor'].merge(tabela_dict['Supervisor'],on='ID Equipe',how='inner')[['ID Vendedor', 'Vendedor', 'Nome Resumido', 'Equipe', 'E-mail',
       'Categoria','Supervisor', 'Email Sup','Gerente', 'Email Gerente']]

In [17]:
tabela_dict['Meta']=tabela_dict['Meta'].loc[(tabela_dict['Meta']['mes_ref'].dt.year==data_atual.year)&(tabela_dict['Meta']['mes_ref'].dt.month==data_atual.month)]

In [18]:
tabela_dict['Vendedor']=tabela_dict['Vendedor'].merge(tabela_dict['Meta'],left_on='ID Vendedor',right_on='cd_vend',how='left')[['ID Vendedor', 'Vendedor', 'Nome Resumido', 'Equipe', 'E-mail',
       'Categoria','Supervisor', 'Email Sup','Gerente', 'Email Gerente','FATUVL']]

tabela_dict['Vendedor'].rename(columns={'FATUVL':'Meta R$'},inplace=True)

In [19]:
grpvenda_df=vendas_df[['ID Vendedor','Total Geral']].loc[(vendas_df['Data de Criação'].dt.year==data_atual.year)&((vendas_df['Data de Criação'].dt.month==data_atual.month))].groupby(['ID Vendedor'],as_index=False).agg({'Total Geral':'sum'})

In [20]:
grppedido_df=vendas_df[['ID Vendedor','Status do Pedido','Numero do Pedido']].loc[(vendas_df['Data de Criação'].dt.year==data_atual.year)&((vendas_df['Data de Criação'].dt.month==(data_atual.month)))].groupby(['ID Vendedor','Status do Pedido'],as_index=False).agg({'Numero do Pedido':'count'})

grppedido_df.rename(columns={'Numero do Pedido':'Pedido'},inplace=True)

grppedido_df.loc[~grppedido_df['Status do Pedido'].isin(['FATURADO','EM ABERTO']),'Pedido']=grppedido_df['Pedido']*-1

grppedido_df=grppedido_df[['ID Vendedor','Pedido']].groupby(['ID Vendedor'],as_index=False).sum()

In [21]:
grpatendimento_df=vendas_df[['ID Vendedor','ID Cliente','Status do Pedido','Numero do Pedido']].loc[(vendas_df['Data de Criação'].dt.year==data_atual.year)&((vendas_df['Data de Criação'].dt.month==(data_atual.month)))].groupby(['ID Vendedor','ID Cliente','Status do Pedido'],as_index=False).agg({'Numero do Pedido':'count'})

grpatendimento_df=grpatendimento_df[['ID Vendedor','Status do Pedido','ID Cliente']].groupby(['ID Vendedor','Status do Pedido'],as_index=False).count()

grpatendimento_df.rename(columns={'ID Cliente':'Atendimento'},inplace=True)

grpatendimento_df.loc[~grpatendimento_df['Status do Pedido'].isin(['FATURADO','EM ABERTO']),'Atendimento']=grpatendimento_df['Atendimento']*-1

grpatendimento_df=grpatendimento_df[['ID Vendedor','Atendimento']].groupby(['ID Vendedor'],as_index=False).sum()

In [22]:
tabela_dict['Vendedor'].columns

Index(['ID Vendedor', 'Vendedor', 'Nome Resumido', 'Equipe', 'E-mail',
       'Categoria', 'Supervisor', 'Email Sup', 'Gerente', 'Email Gerente',
       'Meta R$'],
      dtype='object')

In [23]:
tabela_dict['Vendedor']=tabela_dict['Vendedor'].merge(grpvenda_df,on='ID Vendedor',how='left')[['ID Vendedor', 'Vendedor', 'Nome Resumido', 'Equipe', 'E-mail',
       'Categoria', 'Supervisor', 'Email Sup', 'Gerente', 'Email Gerente','Meta R$','Total Geral']]

In [24]:
tabela_dict['Vendedor']=tabela_dict['Vendedor'].merge(grppedido_df,on='ID Vendedor',how='left')[['ID Vendedor', 'Vendedor', 'Nome Resumido', 'Equipe', 'E-mail',
       'Categoria', 'Supervisor', 'Email Sup', 'Gerente', 'Email Gerente','Meta R$','Total Geral','Pedido']]

In [25]:
tabela_dict['Vendedor']=tabela_dict['Vendedor'].merge(grpatendimento_df,on='ID Vendedor',how='left')[['ID Vendedor', 'Vendedor', 'Nome Resumido', 'Equipe', 'E-mail',
       'Categoria', 'Supervisor', 'Email Sup', 'Gerente', 'Email Gerente','Meta R$','Total Geral','Pedido','Atendimento']]

In [26]:
tabela_dict['Calendario']['Feriado']=tabela_dict['Calendario'].apply(lambda info: tabela_dict['Feriado']['Mês Base'].loc[tabela_dict['Feriado']['Mês Base']==info['Mês Base']].count(),axis=1)

tabela_dict['Calendario']['Dias Úteis']=tabela_dict['Calendario'].apply(lambda info: 0 if info['Feriado']==1 else info['Dias Úteis'],axis=1)

tabela_dict['Calendario'].drop(columns=['Feriado'],inplace=True)

dias_uteis=tabela_dict['Calendario']['Dias Úteis'].loc[(tabela_dict['Calendario']['Data'].dt.year==data_atual.year)&(tabela_dict['Calendario']['Data'].dt.month==data_atual.month)].sum()

tabela_dict['Calendario']['Data']=pd.to_datetime(tabela_dict['Calendario']['Data'].dt.date)

data_min=tabela_dict['Calendario']['Data'].loc[(tabela_dict['Calendario']['Data'].dt.year==data_atual.year)&(tabela_dict['Calendario']['Data'].dt.month==data_atual.month)].min()

data_max=datetime.now()

dias_trabalhados=tabela_dict['Calendario']['Dias Úteis'].loc[(tabela_dict['Calendario']['Data'].dt.year==data_atual.year)&(tabela_dict['Calendario']['Data'].dt.month==data_atual.month)&(tabela_dict['Calendario']['Data'].between(data_min,data_max))].sum()


In [27]:
tabela_dict['Vendedor']['Projeção R$']=tabela_dict['Vendedor'].apply(lambda info: round(((info['Total Geral']/dias_trabalhados)*dias_uteis),2),axis=1)

In [28]:
tabela_dict['Vendedor']['Diferença R$']=tabela_dict['Vendedor'].apply(lambda info: info['Meta R$']-info['Total Geral'],axis=1)

tabela_dict['Vendedor']['% Reliazado']=tabela_dict['Vendedor'].apply(lambda info: round((info['Total Geral']/info['Meta R$']),4)*100 if info['Meta R$']!=0 else 0,axis=1)

tabela_dict['Vendedor']

,ID Vendedor,Vendedor,Nome Resumido,Equipe,E-mail,Categoria,Supervisor,Email Sup,Gerente,Email Gerente,Meta R$,Total Geral,Pedido,Atendimento,Projeção R$,Diferença R$,% Reliazado
0,604,CLIENTES SEM ATENDIMENTO,CLIENTES ATENDIMENTO,GERÊNCIA,COMPRAS@DEMARCHIBRASIL.COM.BR,AVULSO,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,JULIODEL,JULIO C DELFINO,JULIO DELFINO,GERÊNCIA,,CLT,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,NaN,16620.9,218.0,71.0,182829.9,NaN,NaN
2,ROGERIOF,ROGERIO FELIPIM,ROGERIO FELIPIM,GERÊNCIA,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,CLT,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,VERBAE1,VERBAE1,VERBAE1,GERÊNCIA,None,None,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ACLAUDIA,ANA CLAUDIA DA SILVA,ANA SILVA,EQUIPE 8 (ATEND),,CLT,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,RELUCINE,LUCINEIDE RODRIGUES DO NASC 30131201859,LUCINEIDE 30131201859,EQUIPE RE,,PJ,CLIENTES ATENDIMENTO,COMPRAS@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,REMARCOS,MARCOS GONLVES 09059684842,MARCOS 09059684842,EQUIPE RE,,PJ,CLIENTES ATENDIMENTO,COMPRAS@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,REMIQUEI,MIQUEIAS RODRIGO DE OLIVEIRA AMORIM,MIQUEIAS AMORIM,EQUIPE RE,,PJ,CLIENTES ATENDIMENTO,COMPRAS@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103,REMYRELA,MYRELLA BRITO DEL FREU,MYRELLA FREU,EQUIPE RE,MYRELLAFREU@HOTMAIL.COM,PJ,CLIENTES ATENDIMENTO,COMPRAS@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
tabela_dict['Venda'].loc[(tabela_dict['Venda']['Tipo de Vendedor']=='CD_VEND')&(tabela_dict['Venda']['Tipo de Data']=='DT_FATURAMENTO')&(tabela_dict['Venda']['Tipo de Peso']=='PESO_BRT')&(tabela_dict['Venda']['Tipo de Operação']=='VENDAS')]

,ID Empresa,Numero do Pedido,NFe,ID Cliente,Tipo de Vendedor,ID Vendedor,Tipo de Pedido,Tipo de Operação,Tabela de Preço,Tipo de Data,Data de Criação,Status do Pedido,ID NFe,SKU,Seq,Qtde,Unid. VDA,Qtde VDA,Valor VDA,Total Geral,Custo da Útima Entrada,Tipo de Peso,Peso,Preço da Tabela,% Desconto,% Comissão,Comissão R$
4,1,319221,864398.0,25354,CD_VEND,LEONELM,VENDA DEMARCHI,VENDAS,TABELA DE NEGOCIAÇÃO,DT_FATURAMENTO,2021-10-28,DEVOLUÇÃO PARCIAL,352573.0,730,5,-10.0,UN,-10.0,13.13,-131.30,NaN,PESO_BRT,-12.60,NaN,NaN,NaN,NaN
8,1,325841,870296.0,24396,CD_VEND,FABIANO,VENDA DEMARCHI,VENDAS,TABELA DE PREÇO,DT_FATURAMENTO,2021-05-31,DEVOLUÇÃO,359313.0,1113,1,-120.0,CX,-10.0,124.32,-1243.20,-952.37,PESO_BRT,-86.40,11.21,-0.0758,0.005,-6.22
16,1,325889,870314.0,10872,CD_VEND,JOSENUNE,VENDA DEMARCHI,VENDAS,TABELA DE PREÇO,DT_FATURAMENTO,2021-05-31,DEVOLUÇÃO,359332.0,470,2,-15.0,UN,-15.0,11.00,-165.00,-126.00,PESO_BRT,-18.90,13.22,-0.1679,0.010,-1.65
20,1,325889,870314.0,10872,CD_VEND,JOSENUNE,VENDA DEMARCHI,VENDAS,TABELA DE PREÇO,DT_FATURAMENTO,2021-05-31,DEVOLUÇÃO,359332.0,469,1,-25.0,UN,-25.0,11.20,-280.00,-216.01,PESO_BRT,-31.50,11.31,-0.0097,0.010,-2.80
24,1,325978,870342.0,8503,CD_VEND,FABIOARA,VENDA DEMARCHI,VENDAS,TABELA DE PREÇO,DT_FATURAMENTO,2021-05-31,DEVOLUÇÃO,359360.0,9403,1,-24.0,UN,-24.0,17.51,-420.24,-297.60,PESO_BRT,-48.00,17.51,0.0000,0.050,-21.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2030950,1,403220,937641.0,20549,CD_VEND,RODRIGOA,VENDA DEMARCHI,VENDAS,TABELA DE PREÇO,DT_FATURAMENTO,2022-06-02,FATURADO,446866.0,9550,2,22.0,CX,1.0,202.62,202.62,138.72,PESO_BRT,8.80,9.21,0.0000,0.010,2.03
2030958,1,403220,937641.0,20549,CD_VEND,RODRIGOA,VENDA DEMARCHI,VENDAS,TABELA DE PREÇO,DT_FATURAMENTO,2022-06-02,FATURADO,446866.0,1115,4,16.0,CX,1.0,204.95,204.95,156.95,PESO_BRT,9.60,13.86,0.0758,0.010,2.05
2030966,1,403220,937641.0,20549,CD_VEND,RODRIGOA,VENDA DEMARCHI,VENDAS,TABELA DE PREÇO,DT_FATURAMENTO,2022-06-02,FATURADO,446866.0,895,1,20.0,CX,1.0,184.00,184.00,120.00,PESO_BRT,6.10,10.62,0.1337,0.010,1.84
2030974,1,403220,937641.0,20549,CD_VEND,RODRIGOA,VENDA DEMARCHI,VENDAS,TABELA DE PREÇO,DT_FATURAMENTO,2022-06-02,FATURADO,446866.0,405,7,3.0,KG,3.0,47.17,141.51,93.74,PESO_BRT,3.15,51.04,0.0758,0.010,1.42


In [30]:
tabela_dict['Carteira'].columns

Index(['ID Cliente', 'CNPJ', 'Caracter CNPJ', 'Razão Social', 'Nome Fantasia',
       'Status do Cliente', 'Matriz', 'Segmento', 'Canal', 'Tabela',
       'Condição de Pagto', 'Prazo', 'Forma Pagto', 'Crédito', 'Tributação',
       'CEP', 'Endereço', 'Bairro', 'Cidade', 'UF', 'Numero', 'Complemento',
       'Latitude', 'Longitude', 'Região', 'DDD', 'Contato', 'ID Vendedor',
       'Nome', 'Nome Resumido', 'E-mail', 'Categoria', 'Supervisor',
       'Email Sup', 'Gerente', 'Email Gerente', 'Principal', 'Primeira Compra',
       'Última Compra', 'Seq', 'Dias', 'Total Geral', 'Pedido', 'Comprou'],
      dtype='object')

In [81]:
emails={'E-mail':tabela_dict['Carteira']['E-mail'].loc[tabela_dict['Carteira']['E-mail']!=''].unique().tolist(),
         
          'Email Sup':tabela_dict['Carteira']['Email Sup'].loc[tabela_dict['Carteira']['Email Sup']!=''].unique().tolist()
         
         }

colunas={'E-mail':'Email Sup','Email Sup':'Email Gerente'}

tabela_dict['Carteira'].sort_values('Dias',ascending=False,inplace=True)

for coluna,email in emails.items():
    
    for env in email:
        
        email_resp=tabela_dict['Carteira'][colunas[coluna]].loc[tabela_dict['Carteira'][coluna]==env].unique().tolist()
                        
        if(coluna=='E-mail'):
            
            nome=tabela_dict['Carteira']['Nome Resumido'].loc[tabela_dict['Carteira'][coluna]==env].unique().tolist()
            
            arq=(f'{nome[0]}.xlsx')
            
            tabela_dict['Carteira'].loc[tabela_dict['Carteira'][coluna]==env].to_excel(arq,index=False,encoding='ISO-8859-1')            
            
            pass
        
        else:
            
            nome=tabela_dict['Carteira']['Supervisor'].loc[tabela_dict['Carteira'][coluna]==env].unique().tolist()
            
            arq=(f'{nome[0]}.xlsx')
            
            tabela_dict['Carteira'].loc[tabela_dict['Carteira'][coluna]==env].to_excel(arq,index=False,encoding='ISO-8859-1')
                        
            pass
                       
        pass
                
    pass

print('Arquivo gerado')

Arquivo gerado


In [87]:
pd.set_option('float_format','{:.2f}'.format)

tabela_dict['Carteira'].sort_values('Total Geral',ascending=False)

,ID Cliente,CNPJ,Caracter CNPJ,Razão Social,Nome Fantasia,Status do Cliente,Matriz,Segmento,Canal,Tabela,Condição de Pagto,Prazo,Forma Pagto,Crédito,Tributação,CEP,Endereço,Bairro,Cidade,UF,Numero,Complemento,Latitude,Longitude,Região,DDD,Contato,ID Vendedor,Nome,Nome Resumido,E-mail,Categoria,Supervisor,Email Sup,Gerente,Email Gerente,Principal,Primeira Compra,Última Compra,Seq,Dias,Total Geral,Pedido,Comprou
2357,23068,15621463001110,14,NATURAL DA TERRA COMERCIO VAREJISTA HORTIFRUTT...,NATURAL DA TERRA,ATIVO,NATURAL DA TERRA,5 A 9 CHK,AS,TABELA DE PREÇO,DEPOSITO BANCARIO 30 DIAS,30.00,DEP CONTA,RESTRIÇÃO INTERNA,PESSOA JURIDICA - CONTRI NORMAL,5305000.00,"AVENIDA IMPERATRIZ LEOPOLDINA, 244 - GALPAO LA...",VILA LEOPOLDINA,SÃO PAULO,SP,244,GALPAO LADO B LOJA 1B,NaN,NaN,JAGUARE/LEOPOLDINA,11,11111111.00,DOUGLASM,DOUGLAS MARTINS TEDESCHI,DOUGLAS TEDESCHI,DOUGLASMARTINS@DEMARCHIBRASIL.COM.BR,CLT,DOUGLAS TEDESCHI,DOUGLASMARTINS@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,SIM,2020-02-27,2022-05-31,1,2,372970.52,390,NÃO
6,5398,62891254000102,14,AGRO COMERCIAL BONFIM PAULISTA LTDA EPP,AGRO BONFIM,ATIVO,None,DISTRIBUIDOR,AS,DISTRIBUIDORES,A VISTA,1.00,DINHEIRO,NORMAL,PESSOA JURIDICA - CONTRI NORMAL,5316900.00,AVENIDA DOUTOR GASTAO VIDIGAL,VILA LEOPOLDINA,SÃO PAULO,SP,1946,None,NaN,NaN,JAGUARE/LEOPOLDINA,11,36444169.00,LOJA,LOJA,LOJA,,CLT,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,SIM,2018-03-01,2022-06-01,1,1,290194.20,3154,SIM
2272,20867,56527062003550,14,SUPERMERCADO HIROTA LT,HIROTA FOOD CD,ATIVO,HIROTA,5 A 9 CHK,AS,TABELA DE PREÇO,BOLETO BANCARIO 28 DIAS,28.00,BANCO,NORMAL,PESSOA JURIDICA - CONTRI NORMAL,5314000.00,"AVENIDA DOUTOR GASTÃO VIDIGAL - ATÉ 2001/2002,...",VILA LEOPOLDINA,SÃO PAULO,SP,1946,,NaN,NaN,JAGUARE/LEOPOLDINA,None,NaN,DOUGLASM,DOUGLAS MARTINS TEDESCHI,DOUGLAS TEDESCHI,DOUGLASMARTINS@DEMARCHIBRASIL.COM.BR,CLT,DOUGLAS TEDESCHI,DOUGLASMARTINS@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,SIM,2020-08-04,2022-05-26,1,7,262705.72,60,NÃO
2,2145,02357205000154,14,CECILIA TSUYACO ARAKI SILVA,CECILIA TSUYACO ARAK,ATIVO,None,DISTRIBUIDOR,AS,DISTRIBUIDORES,BOLETO BANCARIO 21 DIAS,21.00,None,NORMAL,PESSOA JURIDICA - CONTRI NORMAL,5314000.00,AVENIDA DR. GASTAO VIDIGAL,VILA LEOPOLDINA,SÃO PAULO,SP,1946,None,NaN,NaN,JAGUARE/LEOPOLDINA,11,38326362.00,LOJA,LOJA,LOJA,,CLT,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,SIM,2018-03-01,2022-06-01,1,1,197152.13,1554,SIM
6499,24008,71676316002190,14,SUPERMERCADO MAMBO LTDA,MAMBO CD,ATIVO,MAMBO,10 OU + CHK,AS,TABELA DE PREÇO,DEPOSITO BANCARIO 45 DIAS,45.00,DEP CONTA,NORMAL,PESSOA JURIDICA - CONTRI NORMAL,6612000.00,"VIA DE ACESSO JOÃO DE GÓES, 1701 - BLOCO B RUA...",JARDIM ALVORADA,JANDIRA,SP,1701,BLOCO B RUA A CAMERA 07,NaN,NaN,JANDIRA,11,35073500.00,DOUGLASM,DOUGLAS MARTINS TEDESCHI,DOUGLAS TEDESCHI,DOUGLASMARTINS@DEMARCHIBRASIL.COM.BR,CLT,DOUGLAS TEDESCHI,DOUGLASMARTINS@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,SIM,2021-08-27,2022-05-12,1,21,177840.70,370,NÃO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12807,26912,23245933000144,14,N & H COMERCIO DE ALIMENTOS LTDA,SUBWAY CAMBUCI,ATIVO,None,FAST FOOD/REDES,FS,TABELA DE PREÇO,A VISTA,1.00,DINHEIRO,NORMAL,PESSOA JURIDICA - CONTRI SIMPLES,1519000.00,"RUA DO LAVAPÉS, 1128",CAMBUCI,SÃO PAULO,SP,1128,,NaN,NaN,LIBERDADE,11,32079577.00,JULIODEL,JULIO C DELFINO,JULIO DELFINO,,CLT,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,SIM,2021-12-26,2021-12-26,1,158,0.00,0,NÃO
12715,26819,22471212000190,14,SUB T1 LANCHONETE LTDA,SUBGRUAIRPORTX,ATIVO,None,FAST FOOD/REDES,FS,TABELA DE PREÇO,A VISTA,NaN,DINHEIRO,NORMAL,PESSOA JURIDICA - CONTRI S

In [98]:
tabela_dict['Vendedor'].loc[~tabela_dict['Vendedor']['Equipe'].str.contains('120')]

,ID Vendedor,Vendedor,Nome Resumido,Equipe,E-mail,Categoria,Supervisor,Email Sup,Gerente,Email Gerente,Meta R$,Total Geral,Pedido,Atendimento,Projeção R$,Diferença R$,% Reliazado
0,604,CLIENTES SEM ATENDIMENTO,CLIENTES ATENDIMENTO,GERÊNCIA,COMPRAS@DEMARCHIBRASIL.COM.BR,AVULSO,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,JULIODEL,JULIO C DELFINO,JULIO DELFINO,GERÊNCIA,,CLT,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,NaN,16620.90,218.00,71.00,182829.90,NaN,NaN
2,ROGERIOF,ROGERIO FELIPIM,ROGERIO FELIPIM,GERÊNCIA,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,CLT,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,VERBAE1,VERBAE1,VERBAE1,GERÊNCIA,None,None,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ACLAUDIA,ANA CLAUDIA DA SILVA,ANA SILVA,EQUIPE 8 (ATEND),,CLT,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,RELUCINE,LUCINEIDE RODRIGUES DO NASC 30131201859,LUCINEIDE 30131201859,EQUIPE RE,,PJ,CLIENTES ATENDIMENTO,COMPRAS@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,REMARCOS,MARCOS GONLVES 09059684842,MARCOS 09059684842,EQUIPE RE,,PJ,CLIENTES ATENDIMENTO,COMPRAS@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,REMIQUEI,MIQUEIAS RODRIGO DE OLIVEIRA AMORIM,MIQUEIAS AMORIM,EQUIPE RE,,PJ,CLIENTES ATENDIMENTO,COMPRAS@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103,REMYRELA,MYRELLA BRITO DEL FREU,MYRELLA FREU,EQUIPE RE,MYRELLAFREU@HOTMAIL.COM,PJ,CLIENTES ATENDIMENTO,COMPRAS@DEMARCHIBRASIL.COM.BR,ROGERIO FELIPIM,ROGERIO.FELIPIM@DEMARCHIBRASIL.COM.BR,NaN,NaN,NaN,NaN,NaN,NaN,NaN
